In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [4]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
features = pd.read_csv('data_set.csv', low_memory = False)

In [5]:
print(training_labels.shape)
training_labels = pd.merge(features, training_labels, on='person', how='inner')
print(training_labels.shape)
training_labels.head(2)

(19414, 2)
(19414, 191)


,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,suma,promedio,maximo,minimo,Unnamed: 0,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo,label
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,121083102878903,2.018052e+13,20180517162837,20180517122747,0.0,1.617163,4.851488,1.859879,1.300103,0
1,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,343068993055724,2.018053e+13,20180529003807,20180529002747,4.0,1.000000,4.000000,1.000000,1.000000,0


In [6]:
print(labels_predict.shape)
labels_predict = pd.merge(features, labels_predict, on='person', how='inner')
print(labels_predict.shape)
labels_predict.head(2)

(19415, 1)
(19415, 190)


,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,mes_May,suma,promedio,maximo,minimo,Unnamed: 0,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo
0,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,448.0,9040871528715832,2.018052e+13,20180531195203,20180503220829,1.0,1.490856,557.580165,2.209395,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,0.0,201803261454582,2.018033e+13,20180326145658,20180326145111,2.0,1.219303,3.657909,1.357806,1.0


In [7]:
labels = training_labels['label']
features = training_labels.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  17472 Test:  1942


## Classification and Regression Trees

In [8]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
print(model)

expected = y_test
predicted = model.predict(x_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
             precision    recall  f1-score   support

          0       0.96      0.94      0.95      1851
          1       0.13      0.18      0.15        91

avg / total       0.92      0.91      0.91      1942

[[1748  103]
 [  75   16]]


In [9]:
set_test = labels_predict
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])
set_test = set_test.fillna(0)

In [10]:
preds_kaggel_knn = model.predict_proba(set_test)

In [11]:
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 2 columns):
0    19415 non-null float64
1    19415 non-null float64
dtypes: float64(2)
memory usage: 303.4 KB


In [12]:
df_kaggel_knn = pd.DataFrame(df_kaggel_knn[1])

In [13]:
df_kaggel_knn["person"] = personas
df_kaggel_knn.rename(columns = {1: 'label'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['person','label']]

In [14]:
df_kaggel_knn.head()

,person,label
0,00091926,0.0
1,00091a7a,0.0
2,000ba417,0.0
3,000e4d9e,0.0
4,000e619d,0.0


In [16]:
df_kaggel_knn.to_csv('ResultadosCART.csv',index=False)